In [2]:
import numpy as np
import cv2
import os
import sys
from tensorflow.keras.utils import Progbar

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Input, Dropout, Lambda, Conv2D, Conv2Dtranspose

ImportError: cannot import name 'Conv2Dtranspose' from 'tensorflow.keras.layers' (C:\Users\user\anaconda3\lib\site-packages\tensorflow\keras\layers\__init__.py)

In [7]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, x_data, y_data, transform_img=None, transform_mask=None):
        self.transform_img = transform_img
        self.transform_mask = transform_mask
        
        self.data = np.transpose(x_data, (0, 3, 1, 2))
        self.targets = np.transpose(y_data, (0, 3, 1, 2))
        
    def __len__(self):
        return self.targets.shape[0]
    
    def __getitem__(self, index):
        x=self.data[index]
        y=self.targets[index]
        
        if self.transform_img :
            seed = np.random.randint(2147483647)
            random.seed(seed)
            
            x=Image.fromarray(np.squeeze(self.targets[index].stype(np.uint8).transpose(1, 2, 0)))
            y = self.transform_mask(y)
        return x, y